In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
con = sqlite3.connect('data/db1')
cur = con.cursor()
def select(sql):
    return pd.read_sql(sql, con)


In [8]:
# sql ='''insert into book(title, author, price, amount)
# VALUES ('Стихотворения и поэмы', 'Есенин С.А.', 650.00, 15)'''
# cur.execute(sql)

In [11]:
sql ='''
select t.title from book t'''
select(sql)

,card_number,coffee_price,count(1),rank() over (PARTITION BY t.coffee_price order by t.discount )
0,12.0,100,1,1
1,32.0,120,1,1
2,8.0,120,2,1
3,20.0,120,2,19
4,17.0,150,2,1
...,...,...,...,...
158,19.0,130,25,17
159,NaN,100,57,19
160,NaN,120,58,1
161,NaN,150,74,25


In [17]:
sql ='''select t.title, t.author, t.amount, round(t.price * 0.7, 2) as pack from book t'''
select(sql)

,title,author,amount,pack
0,Мастер и Маргарита,Булгаков М.А.,3,469.69
1,Белая гвардия,Булгаков М.А.,5,378.35
2,Идиот,Достоевский Ф.М.,10,322.00
3,Братья Карамазовы,Достоевский Ф.М.,2,559.31


In [21]:
sql ='''select t.*, 
 case
    when t.author='Булгаков М.А.' then round(t.price*1.1, 2)
    when t.author='Есенин С.А.' then round(t.price*1.05, 2)
    else round(t.price,2) end as new_price
 from book t'''
select(sql)

,book_id,title,author,price,amount,new_price
0,1,Мастер и Маргарита,Булгаков М.А.,670.99,3,738.09
1,2,Белая гвардия,Булгаков М.А.,540.50,5,594.55
2,3,Идиот,Достоевский Ф.М.,460.00,10,460.00
3,4,Братья Карамазовы,Достоевский Ф.М.,799.01,2,799.01


In [23]:
sql ='''select t.*,
 case
     when t.amount=10 then 20
     when t.amount>4 then 15
     else t.amount end as alians
 from book t'''
select(sql)

,book_id,title,author,price,amount,alians
0,1,Мастер и Маргарита,Булгаков М.А.,670.99,3,3
1,2,Белая гвардия,Булгаков М.А.,540.50,5,15
2,3,Идиот,Достоевский Ф.М.,460.00,10,20
3,4,Братья Карамазовы,Достоевский Ф.М.,799.01,2,2


In [28]:
sql ='''select title, author, price, amount from book
where (price<500 or price>600) and price*amount>5000'''
select(sql)

,title,author,price,amount
0,Стихотворения и поэмы,Есенин С.А.,650,15


In [3]:
sql ='''select title, author from book
where price between 540.50 and 800 and (amount=2 or amount=3 or amount=5 or amount=7)'''
select(sql)

,title,author
0,Мастер и Маргарита,Булгаков М.А.
1,Белая гвардия,Булгаков М.А.
2,Братья Карамазовы,Достоевский Ф.М.


In [11]:
sql ='''select title, author from book
where title like '%_ %_' and author like '%С.%' '''
select(sql)

,title,author
0,Стихотворения и поэмы,Есенин С.А.


In [14]:
sql ='''select author, title from book
where amount between 2 and 14
order by author desc, title'''
select(sql)

,author,title
0,Достоевский Ф.М.,Братья Карамазовы
1,Достоевский Ф.М.,Идиот
2,Булгаков М.А.,Белая гвардия
3,Булгаков М.А.,Мастер и Маргарита


## GROUP BY

In [15]:
sql ='''select distinct amount from book'''
select(sql)

,amount
0,3
1,5
2,10
3,2
4,15


In [17]:
sql ='''select author as Автор,
 count(title) as Различных_книг,
 sum(amount) as Количество_экземпляров
 from book
 group by Автор'''
select(sql)

,Автор,Различных_книг,Количество_экземпляров
0,Булгаков М.А.,2,8
1,Достоевский Ф.М.,2,12
2,Есенин С.А.,1,15


In [18]:
sql ='''select author,
 min(price) as Минимальная_цена,
 max(price) as Максимальная_цена,
 avg(price) as Средняя_цена
 from book
 group by author'''
select(sql)

,author,Минимальная_цена,Максимальная_цена,Средняя_цена
0,Булгаков М.А.,540.5,670.99,605.745
1,Достоевский Ф.М.,460.0,799.01,629.505
2,Есенин С.А.,650.0,650.00,650.000


In [22]:
sql ='''select author,
 sum(price*amount) as Стоимость,
 round((sum(price*amount)*18/100) / (1+18/100), 2) as НДС,
 round(sum(price*amount) / (1+18/100), 2) as Стоимость_без_НДС
 from book
 group by author'''
select(sql)

,author,Стоимость,НДС,Стоимость_без_НДС
0,Булгаков М.А.,4715.47,848.78,4715.47
1,Достоевский Ф.М.,6198.02,1115.64,6198.02
2,Есенин С.А.,9750.00,1755.00,9750.00


In [24]:
sql ='''select round(min(price),2) as Минимальная_цена,
 round(max(price),2) as Максимальная_цена,
 round(avg(price),2) as Средняя_цена
 from book'''
select(sql)

,Минимальная_цена,Максимальная_цена,Средняя_цена
0,460.0,799.01,624.1


In [26]:
sql ='''select
 round(avg(price),2) as Средняя_цена,
 round(sum(price*amount),2) as Стоимость
 from book
 where amount between 5 and 14'''
select(sql)

,Средняя_цена,Стоимость
0,500.25,7302.5


In [9]:
sql ='''select t.* from book t'''
select(sql)

,book_id,title,author,price,amount
0,1,Мастер и Маргарита,Булгаков М.А.,670.99,3
1,2,Белая гвардия,Булгаков М.А.,540.50,5
2,3,Идиот,Достоевский Ф.М.,460.00,10
3,4,Братья Карамазовы,Достоевский Ф.М.,799.01,2
4,5,Стихотворения и поэмы,Есенин С.А.,650.00,15
